In [2]:
import numpy as np
import scipy.stats
import os
import re

import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.multitest

import csv
import pandas as pd
import matplotlib.pyplot as plt



In [3]:
# Define file paths
data_dir = '../data'
pupil_dir = '../data/SingleTrialPupilData'
eeg_dir = '../data/SingleTrialEEGData'
plot_dir = '../plots'

include:
- theta: frontal & central
- alpha: central & parietal
- beta: central & parietal

exclude:
-  trials with no eeg
-  trials with no pain data
    - P05: block 3 & 4
    - P14: last trials of block 3 & all of block 4

## Model Fitting

Skip if loading previously fitted data

In [4]:
files_pupil = os.listdir(pupil_dir)
files_eeg = os.listdir(eeg_dir)

In [5]:
all_tonic_sides = pd.read_csv(os.path.join(data_dir, 'tonicpainsides.csv'))

In [6]:
def function(lr, file, V_0):

    """
    Updates and computes the correlation between V values and mean EEG frequency data.
    
    Parameters:
        - lr (float): 
            Learning rate for updating V values.
        - file (str): 
            Filename of the pupil data CSV.
        - V_0 (list or array): 
            Initial V values.
    
    Returns:
        float: Negative Pearson correlation coefficient between updated V values and mean EEG frequencies.
    """
    
    # Extract participant identifier from the filename
    res = re.search('(...)_PupilDiameterProcessed', file)
    participant = res.group(1)

    # Find corresponding EEG file for the participant
    match_eeg = [s for s in files_eeg if participant in s]

    # Load pupil and EEG data
    df_pupil = pd.read_csv(os.path.join(pupil_dir, file))
    df_eeg = pd.read_csv(os.path.join(eeg_dir, match_eeg[0]))

    # Calculate the cumulative number of trials up to each block
    block_1 = np.sum(df_eeg['Block'] == 1)
    block_2 = block_1 + np.sum(df_eeg['Block'] == 2)
    block_3 = block_2 + np.sum(df_eeg['Block'] == 3)
    block_4 = block_3 + np.sum(df_eeg['Block'] == 4)

    # Recode 'side' as numerical values
        # 'side' refers to the direction of the rock
    df_eeg['side'] = (df_eeg['type']== 'left') * 0 + \
        (df_eeg['type']== 'middle') * 1 + \
        (df_eeg['type']== 'right') * 2

     # Compute mean frequency across specified frequency bands
    df_eeg['mean'] = np.mean(df_eeg.loc[:, freq_include], axis = 1)

    # Initialize V values and array to store V values for each trial
    V = np.array((V_0))                 
    all_V = np.zeros((np.shape(df_eeg)[0] + 1, 3))
    all_V[0, :] = V
                           
    # Calculate V values for each trial across blocks
    for block in [1,2,3,4]:

        # Skip block 3 for participant 5, as pain values are missing
        if participant == 'P05' and (block == 3 or block == 4):
            continue
        if participant == 'P14' and block == 4:
            continue
        
        # Get data for block
        this_block = df_eeg[df_eeg['Block'] == block]
        
        for i_t, trial in enumerate(this_block['epoch']):
            if participant == 'P14' and trial>241:
                continue
            # Determine cue side based on 'type'
            if this_block.loc[trial-1, 'type'] == 'left':
                cue_side = 0
            elif this_block.loc[trial-1, 'type'] == 'middle':
                cue_side = 1
            elif this_block.loc[trial-1, 'type'] == 'right':
                cue_side = 2
            else:
                print('Error: Unrecognized cue side.')

            # Determine reward value based on 'pain'
            if this_block.loc[trial-1,'pain'] == 'PCollShock':
                r = 1
            elif this_block.loc[trial-1,'pain'] == 'nPCollNoShock':
                r = 0
            elif this_block.loc[trial-1,'pain'] == 'PCollNoShock':
                r = 0
            elif np.logical_and(np.isnan(this_block.loc[trial-1,'pain']), block == 2 or block == 4):
                r = 0
            else:
                if this_block.loc[trial-1, 'type'] == 'middle':
                    r = 0
                else:
                    r = 1

            # Update V
            V[cue_side] = V[cue_side] * (1 - lr) + lr * r
            all_V[trial, :] = V
   
    # Create a boolean mask for rows where 'Include' column equals 1 (used for filtering EEG data). Change mask to exclude the last trial. This is because, we want to correlate the V-value before the trial to the eeg data
    V_incl = (df_eeg['Include'] == 1)
    V_incl[-1] = False

    # Create a boolean mask for rows where 'Include' column equals 1 (used for filtering EEG data)
    eeg_incl = (df_eeg['Include'] == 1)

    # Exclude the last trials for participant 5 and 14, as pain values are missing
    if participant == 'P05':
        eeg_incl[180::] = False
        V_incl[180::] = False
    elif participant == 'P14':
        eeg_incl[241::] = False
        V_incl[241::] = False

    # Calculate the Pearson correlation coefficient between:
    # 1. The values in `all_V` corresponding to the V-values of the side from which the rock is approaching in that trial.
    # 2. The mean EEG frequencies 
    # for the rows where 'Include' is 1
    correlation = scipy.stats.pearsonr(
            all_V[V_incl, df_eeg.loc[eeg_incl, 'side']],
            df_eeg['mean'].loc[eeg_incl])[0]

    return (-correlation)

In [7]:
# Define boundary constraints for optimization. 
# Lower bound (lb) is set to 0.0 and upper bound (ub) is set to 1.0.
bnds = scipy.optimize.Bounds(lb=0.01, ub=0.99)

# List of frequency variables to include in the optimization or analysis.
# These variables seem represent frequency band for different 
# electrode locations (el1, el2, el3) and timebins (t1, t2, t3).
freq_include = ['Freq_f1_t1_el1', 'Freq_f1_t2_el1', 'Freq_f1_t3_el1',
           'Freq_f1_t1_el2', 'Freq_f1_t2_el2', 'Freq_f1_t3_el2',
           'Freq_f2_t1_el2', 'Freq_f2_t2_el2', 'Freq_f2_t3_el2',
           'Freq_f2_t1_el3', 'Freq_f2_t2_el3', 'Freq_f2_t3_el3',
           'Freq_f3_t1_el2', 'Freq_f3_t2_el2', 'Freq_f3_t3_el2',
           'Freq_f3_t1_el3', 'Freq_f3_t2_el3', 'Freq_f3_t3_el3']


In [8]:
# Intialize DataFrame to save relevant trial data (EEG, V values, participant ID, and congruency)
all_data = pd.DataFrame()

# Intialize DataFrame to save relevant model fitting data (ID, learning rate, maximum correlation)
df_model = pd.DataFrame()

for i_file, file in enumerate(files_pupil):
    print('========= \n File : {} of {}'.format(i_file + 1, len(files_pupil)))
    try:
        # Initialize V Values for RL
        V_0 = np.array((0.3, 0.3, 0.3))

        # Extract the participant ID from the file name
        ID = int(re.split('_PupilDiameterProcessed_ST.csv', file)[0][1::])

        # Extract the tonic pain sides for the participant
        tonic_sides  = all_tonic_sides.loc[all_tonic_sides['ID'] == ID, 'Extn1' : 'Extn2'].values[0].astype('str')
        tonic_sides[np.where(tonic_sides == '1')] = 'left'
        tonic_sides[np.where(tonic_sides == '2')] = 'right'
        
        # Fit the model
        fit_res = scipy.optimize.minimize(function, (0.5, ), args=(file, V_0), method= 'SLSQP', bounds = bnds)
        print(fit_res.message)
        
        # Extract participant identifier from the filename
        res_file = re.search('(...)_PupilDiameterProcessed', file)
        participant = res_file.group(1)

        # Find corresponding EEG file for the participant
        match_eeg = [s for s in files_eeg if participant in s]

        # Load pupil and EEG data
        df_pupil = pd.read_csv(os.path.join(pupil_dir, file))
        df_eeg = pd.read_csv(os.path.join(eeg_dir, match_eeg[0]))


        # Recode 'side' as numerical values
            # 'side' refers to the direction of the rock
        df_eeg['side'] = (df_eeg['type']== 'left') * 0 + \
            (df_eeg['type']== 'middle') * 1 + \
            (df_eeg['type']== 'right') * 2

        # Initialize V values
        V = np.array((V_0))
        all_V = np.zeros((np.shape(df_eeg)[0] + 1, 3))
        all_V[0, :] = V
    
        # Save learning rate from model fit
        lr = fit_res.x
        
        # Loop through each block (1 to 4)
        for block in [1,2,3,4]:
            
            # Skip block 3 for participant 5, as pain values are missing
            if ID == 5 and (block == 3 or block == 4):
                print('Skipping block 3 and 4 for participant 5')
                continue
            if ID == 14 and block == 4:
                print('Skipping block 4 for participant 14')
                continue

            # Extract data for this block
            this_block = df_eeg[df_eeg['Block'] == block]
            
             # Loop through each trial in the block
            for i_t, trial in enumerate(this_block['epoch']):
                
                if ID == 14 and trial>241:
                    print('Skipping trial {} in block 3 for participant 14'.format(trial))
                    continue
                
                # Determine the cue side for the current trial (left, middle, right)
                if this_block.loc[trial-1, 'type'] == 'left':
                    cue_side = 0
                elif this_block.loc[trial-1, 'type'] == 'middle':
                    cue_side = 1
                elif this_block.loc[trial-1, 'type'] == 'right':
                    cue_side = 2
                else:
                    print('Error: Unrecognized cue side.')

                # Determine reward based on 'pain' value
                if this_block.loc[trial-1,'pain'] == 'PCollShock':
                    r = 1
                elif this_block.loc[trial-1,'pain'] == 'nPCollNoShock':
                    r = 0
                elif this_block.loc[trial-1,'pain'] == 'PCollNoShock':
                    r = 0
                elif np.logical_and(np.isnan(this_block.loc[trial-1,'pain']), block == 2 or block == 4):
                    r = 0
                else:
                    print('Error: Unrecognized pain value: {}'.format(this_block.loc[trial-1,'pain']))
                    if this_block.loc[trial-1, 'type'] == 'middle':
                        r = 0
                    else:
                        r = 1
                    
                # Update V based on reward and learning rate
                V[cue_side] = V[cue_side] * (1 - lr) + lr * r
                all_V[trial, :] = V
        
        # Create masks for trials to include
        V_inc = (df_eeg['Include'] == 1)
        V_inc[np.shape(V_inc)[0] + 1] = False # Exclude the last row
        eeg_incl = (df_eeg['Include'] == 1)

        # Exclude the last trials for participant 5 and 14, as pain values are missing
        if participant == 'P05':
            eeg_incl[180::] = False
            V_inc[180::] = False
        elif participant == 'P14':
            eeg_incl[241::] = False
            V_inc[241::] = False

        # Create a DataFrame for the V values for the included trials
        df_V = pd.DataFrame(all_V[
            V_inc, 
            df_eeg.loc[eeg_incl, 'side']],
            columns = ['V'])
                            
        # Create a DataFrame for the participant ID (repeated for each trial)
        df_ID = pd.DataFrame(np.repeat(ID, np.shape(df_V)[0]), columns = ['ID'])
        
        # Filter EEG data for included trials
        df_eeg_inc = df_eeg.loc[eeg_incl, freq_include].reset_index(drop = True)
        
        # Identify congruent and incongruent trials based on block and side information
        cong = np.logical_or(
            np.logical_and(df_eeg['Block'] == 2, df_eeg['type'] == tonic_sides[0]),
            np.logical_and(df_eeg['Block'] == 4, df_eeg['type'] == tonic_sides[1]))
        incong = np.logical_or(
            np.logical_and(df_eeg['Block'] == 2, df_eeg['type'] == tonic_sides[1]),
            np.logical_and(df_eeg['Block'] == 4, df_eeg['type'] == tonic_sides[0]))

        # Create a DataFrame for congruency information (congruent or incongruent on extinction blocks; nan for acquisition blocks)
        df_congr = (np.ones((np.shape(df_eeg)[0])) * np.nan).astype('str')
        df_congr[cong] = "congr" # congruent trials
        df_congr[incong] = "incongr" # incongruent trials
        
        # Filter congruency data for included trials
        df_congr = pd.DataFrame(df_congr[eeg_incl], columns = ['congruency'])
                
        # Concatenate the relevant data (EEG, V values, participant ID, and congruency) into a new row
        new_row = pd.concat([df_eeg_inc,df_V, df_ID, df_congr], axis = 1)
        
        # Append the new row to the overall data
        all_data = pd.concat([all_data, new_row])

        # Append the model fitting data
        df_model = pd.concat([df_model, pd.DataFrame({'ID': ID, 'lr': lr, 'max_corr': -fit_res.fun})])
    
    except:
        # Print error message if something goes wrong during file processing
        print('__something went wrong')

# Reset index of the final DataFrame
all_data.reset_index(drop = True)

print('DONE')

 File : 1 of 26
Optimization terminated successfully
 File : 2 of 26
Optimization terminated successfully
 File : 3 of 26
Optimization terminated successfully
 File : 4 of 26
Optimization terminated successfully
Skipping block 3 and 4 for participant 5
Skipping block 3 and 4 for participant 5
 File : 5 of 26
Optimization terminated successfully
 File : 6 of 26
Optimization terminated successfully
 File : 7 of 26
Optimization terminated successfully
 File : 8 of 26
Optimization terminated successfully
 File : 9 of 26
Optimization terminated successfully
Error: Unrecognized pain value: nan
 File : 10 of 26
Optimization terminated successfully
 File : 11 of 26
Optimization terminated successfully
Error: Unrecognized pain value: nan
Error: Unrecognized pain value: nan
 File : 12 of 26
Optimization terminated successfully
Skipping trial 242 in block 3 for participant 14
Skipping trial 243 in block 3 for participant 14
Skipping trial 244 in block 3 for participant 14
Skipping trial 245 in bl

In [9]:
df_model.to_csv(os.path.join(data_dir, 'eeg_model_fitting_results.csv'), index = False)

In [10]:
all_data.to_csv(os.path.join(data_dir, 'eeg_v_values.csv'), index = False)

## Load Data

In [11]:
# Read csv file into DataFrame
all_data = pd.read_csv(os.path.join(data_dir, 'eeg_v_values.csv'))

## Regression

In [12]:
# Reshape the 'all_data' DataFrame from wide to long format, using 'ID', 'V', and 'congruency' as identifier variables.
# The columns of 'all_data' are melted into two new columns: 'f_t_el' (frequency, timebin & electrode placement information) and 'eeg' (eeg values).
dat = pd.melt(all_data, id_vars = ['ID','V', 'congruency'], var_name = 'f_t_el', value_name = 'eeg')

# Display the reshaped DataFrame 'dat'
dat

,ID,V,congruency,f_t_el,eeg
0,2,3.000000e-01,NaN,Freq_f1_t1_el1,132.765956
1,2,3.000000e-01,NaN,Freq_f1_t1_el1,-15.212086
2,2,6.500000e-01,NaN,Freq_f1_t1_el1,-15.784883
3,2,6.500000e-01,NaN,Freq_f1_t1_el1,15.787904
4,2,3.000000e-01,NaN,Freq_f1_t1_el1,-8.055991
...,...,...,...,...,...
157639,30,9.999010e-71,incongr,Freq_f3_t3_el3,-26.454796
157640,30,9.999990e-69,congr,Freq_f3_t3_el3,-26.478700
157641,30,9.999990e-71,congr,Freq_f3_t3_el3,-39.408142
157642,30,3.000000e-239,NaN,Freq_f3_t3_el3,-2.559981


### EEG ~ V * f_t_el

In [13]:
# Performing an ordinary least squares (OLS) regression analysis, modeling 'eeg' as a function of 'V' and 'f_t_el',including their interaction ('V*f_t_el')
res = smf.ols('eeg ~ V*f_t_el', data = dat).fit()

# Display the summary of the regression results (including coefficients, p-values, etc.).
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    eeg   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     42.83
Date:                Mon, 09 Sep 2024   Prob (F-statistic):          3.01e-291
Time:                        20:08:31   Log-Likelihood:            -8.8350e+05
No. Observations:              157644   AIC:                         1.767e+06
Df Residuals:                  157608   BIC:                         1.767e+06
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     16.0144      0.826     19.387      0.000      14.395      17.633
f_t_el[T.Freq_f1_t1_el2]      -2.4447      1.168     -2.093      0.036      -4.734      -0.155
f_t_el[T.Freq_f1_t2_el1]      -4.0955      1.168     -3.506      0.000      -6.385      -1.806
f_t_el[T.Freq_f1_t2_el2]     -13.3999      1.168    -11.471      0.000     -15.689     -11.110
f_t_el[T.Freq_f1_t3_el1]     -14.0868      1.168    -12.059      0.000     -16.376     -11.797
f_t_el[T.Freq_f1_t3_el2]     -12.7203      1.168    -10.889      0.000     -15.010     -10.431
f_t_el[T.Freq_f2_t1_el2]     -13.1692      1.168    -11.273      0.000     -15.459     -10.880
f_t_el[T.Freq_f2_t1_el3]     -17.3134      1.168    -14.821      0.000     -19.603     -15.024
f_t_el[T.Freq_f2_t2_el2]     -16.5292      1.168    -14.150      0.000     -18.819     -14.240
f_t_el[T.Freq_f2_t2_el3]     -19.8060      1.168    -16.955      0.000     -22.096     -17.516
f_t_el[T.Freq_f2_t3_el2]     -12.1123      1.168    -10.369      0.000     -14.402      -9.823
f_t_el[T.Freq_f2_t3_el3]      -9.2756      1.168     -7.940      0.000     -11.565      -6.986
f_t_el[T.Freq_f3_t1_el2]     -16.0063      1.168    -13.702      0.000     -18.296     -13.717
f_t_el[T.Freq_f3_t1_el3]     -22.3378      1.168    -19.122      0.000     -24.627     -20.048
f_t_el[T.Freq_f3_t2_el2]     -17.2064      1.168    -14.729      0.000     -19.496     -14.917
f_t_el[T.Freq_f3_t2_el3]     -18.0216      1.168    -15.427      0.000     -20.311     -15.732
f_t_el[T.Freq_f3_t3_el2]     -17.6361      1.168    -15.097      0.000     -19.926     -15.347
f_t_el[T.Freq_f3_t3_el3]     -18.0159      1.168    -15.422      0.000     -20.306     -15.726
V                              3.7282      2.016      1.849      0.064      -0.223       7.679
V:f_t_el[T.Freq_f1_t1_el2]    -0.9389      2.851     -0.329      0.742      -6.527       4.649
V:f_t_el[T.Freq_f1_t2_el1]    -0.8284      2.851     -0.291      0.771      -6.416       4.759
V:f_t_el[T.Freq_f1_t2_el2]    -2.6255      2.851     -0.921      0.357      -8.213       2.962
V:f_t_el[T.Freq_f1_t3_el1]    -5.1932      2.851     -1.822      0.069     -10.781       0.395
V:f_t_el[T.Freq_f1_t3_el2]    -4.3262      2.851     -1.517      0.129      -9.914       1.262
V:f_t_el[T.Freq_f2_t1_el2]    -1.6389      2.851     -0.575      0.565      -7.227       3.949
V:f_t_el[T.Freq_f2_t1_el3]     0.8837      2.851      0.310      0.757      -4.704       6.472
V:f_t_el[T.Freq_f2_t2_el2]    -7.6437      2.851     -2.681      0.007     -13.231      -2.056
V:f_t_el[T.Freq_f2_t2_el3]    -8.5027      2.851     -2.982      0.003     -14.091      -2.915
V:f_t_el[T.Freq_f2_t3_el2]   -12.0860      2.851     -4.239      0.000     -17.674      -6.498
V:f_t_el[T.Freq_f2_t3_el3]   -12.6167      

In [14]:
# Extract the p-values from the OLS regression result.
p_values = res.pvalues 

# Perform multiple hypothesis testing correction on the p-values using the Benjamini-Hochberg method (FDR correction), with a significance level (alpha) of 0.05.
_, pvals_corrected, _, _ = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')

# Convert the original and corrected p-values to a DataFrame for better visualization.
dat_p = p_values.to_frame()
dat_p.insert(1, "corrected", pvals_corrected)

# Save the DataFrame with original and corrected p-values to an Excel file.
dat_p.to_excel("output_eeg_V-f_t_el.xlsx")
dat_p

,0,corrected
Intercept,1.232706e-83,4.437740e-82
f_t_el[T.Freq_f1_t1_el2],3.637334e-02,5.237762e-02
f_t_el[T.Freq_f1_t2_el1],4.551529e-04,8.623949e-04
f_t_el[T.Freq_f1_t2_el2],1.899111e-30,5.697334e-30
f_t_el[T.Freq_f1_t3_el1],1.801023e-33,5.894257e-33
f_t_el[T.Freq_f1_t3_el2],1.327821e-27,3.414397e-27
f_t_el[T.Freq_f2_t1_el2],1.822219e-29,5.046146e-29
f_t_el[T.Freq_f2_t1_el3],1.156724e-49,5.948865e-49
f_t_el[T.Freq_f2_t2_el2],1.998211e-45,7.992844e-45
f_t_el[T.Freq_f2_t2_el3],2.022963e-64,2.427556e-63


### EEG ~ V*congruency

In [15]:
# Performing an ordinary least squares (OLS) regression analysis for the extinction blocks, modeling 'eeg' as a function of 'V' and congruency including their interaction.
dat_cong = dat[~(dat['congruency'].values == 'nan')]
res = smf.ols('eeg ~ V*congruency', data = dat_cong).fit()

# Display the summary of the regression results (including coefficients, p-values, etc.).
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    eeg   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     6.005
Date:                Mon, 09 Sep 2024   Prob (F-statistic):           0.000437
Time:                        20:08:32   Log-Likelihood:            -3.5695e+05
No. Observations:               62478   AIC:                         7.139e+05
Df Residuals:                   62474   BIC:                         7.139e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.9369      0.456      2.055      0.040       0.043       1.831
congruency[T.incongr]       1.9362      0.646      2.998      0.003       0.670       3.202
V                          -0.0071      2.314     -0.003      0.998      -4.542       4.528
V:congruency[T.incongr]    -8.3052      3.227     -2.573      0.010     -14.631      -1.979
==============================================================================
Omnibus:                   156525.790   Durbin-Watson:                   1.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       8171530933.989
Skew:                          27.005   Prob(JB):                         0.00
Kurtosis:                    1773.891   Cond. No.                         14.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# Extract the p-values from the OLS regression result.
p_values = res.pvalues

# Perform multiple hypothesis testing correction on the p-values using the Benjamini-Hochberg method (FDR correction), with a significance level (alpha) of 0.05.
_, pvals_corrected, _, _ = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')

# Convert the original and corrected p-values to a DataFrame for better visualization.
dat_p = p_values.to_frame()
dat_p.insert(1, "corrected", pvals_corrected)

# Save the DataFrame with original and corrected p-values to an Excel file.
dat_p.to_excel("output_eeg_V-congruency.xlsx")
dat_p

,0,corrected
Intercept,0.039888,0.053184
congruency[T.incongr],0.002723,0.010890
V,0.997556,0.997556
V:congruency[T.incongr],0.010076,0.020152


### EEG ~ V * f_t_el * congruency

In [17]:
# Performing an ordinary least squares (OLS) regression analysis for the extinction blocks, modeling 'eeg' as a function of 'V', 'f_t_el' and congruency including their interaction.
dat_cong = dat[~(dat['congruency'].values == 'nan')]
res = smf.ols('eeg ~ V*f_t_el*congruency', data = dat_cong).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    eeg   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     7.155
Date:                Mon, 09 Sep 2024   Prob (F-statistic):           6.52e-67
Time:                        20:08:33   Log-Likelihood:            -3.5671e+05
No. Observations:               62478   AIC:                         7.136e+05
Df Residuals:                   62406   BIC:                         7.142e+05
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           11.2151      1.928      5.818      0.000       7.437      14.994
f_t_el[T.Freq_f1_t1_el2]                            -0.5562      2.726     -0.204      0.838      -5.900       4.787
f_t_el[T.Freq_f1_t2_el1]                            -0.9132      2.726     -0.335      0.738      -6.257       4.430
f_t_el[T.Freq_f1_t2_el2]                           -10.2682      2.726     -3.766      0.000     -15.612      -4.925
f_t_el[T.Freq_f1_t3_el1]                           -12.1981      2.726     -4.474      0.000     -17.542      -6.854
f_t_el[T.Freq_f1_t3_el2]                            -8.3150      2.726     -3.050      0.002     -13.659      -2.971
f_t_el[T.Freq_f2_t1_el2]                            -9.2902      2.726     -3.408      0.001     -14.634      -3.947
f_t_el[T.Freq_f2_t1_el3]                           -14.2769      2.726     -5.237      0.000     -19.621      -8.933
f_t_el[T.Freq_f2_t2_el2]                           -12.8266      2.726     -4.705      0.000     -18.170      -7.483
f_t_el[T.Freq_f2_t2_el3]                           -14.9410      2.726     -5.480      0.000     -20.285      -9.597
f_t_el[T.Freq_f2_t3_el2]                            -8.7649      2.726     -3.215      0.001     -14.109      -3.421
f_t_el[T.Freq_f2_t3_el3]                            -5.8616      2.726     -2.150      0.032     -11.205      -0.518
f_t_el[T.Freq_f3_t1_el2]                           -12.5743      2.726     -4.612      0.000     -17.918      -7.231
f_t_el[T.Freq_f3_t1_el3]                           -18.8270      2.726     -6.906      0.000     -24.171     -13.483
f_t_el[T.Freq_f3_t2_el2]                           -13.2570      2.726     -4.863      0.000     -18.601      -7.913
f_t_el[T.Freq_f3_t2_el3]                           -14.6133      2.726     -5.360      0.000     -19.957      -9.270
f_t_el[T.Freq_f3_t3_el2]                           -12.9679      2.726     -4.757      0.000     -18.312      -7.624
f_t_el[T.Freq_f3_t3_el3]                           -14.5562      2.726     -5.339      0.000     -19.900      -9.213
congruency[T.incongr]                                7.8133      2.731      2.861      0.004       2.460      13.166
f_t_el[T.Freq_f1_t1_el2]:congruency[T.incongr]      -3.8168      3.863     -0.988      0.323     -11.387       3.754
f_t_el[T.Freq_f1_t2_el1]:congruency[T.incongr]      -3.0322      3.863     -0.785      0.432     -10.603       4.538
f_t_el[T.Freq_f1_t2_el2]:congruency[T.incongr]      -5.9460      3.863     -1.539      0.124     -13.517       1.625
f_t_el[T.Freq_f1_t3_el1]:congruency[T.incongr]       0.0545      3.863      0.014      0.989      -7.516       7.625
f_t_el[T.Freq_f1_t3_el2]:congruency[T.inc

In [18]:
# Extract the p-values from the OLS regression result.
p_values = res.pvalues

# Perform multiple hypothesis testing correction on the p-values using the Benjamini-Hochberg method (FDR correction), with a significance level (alpha) of 0.05.
_, pvals_corrected, _, _ = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')

# Convert the original and corrected p-values to a DataFrame for better visualization.
dat_p = p_values.to_frame()
dat_p.insert(1, "corrected", pvals_corrected)

# Save the DataFrame with original and corrected p-values to an Excel file.
dat_p.to_excel("output_eeg_V-f_t_el-congruency.xlsx")
dat_p

,0,corrected
Intercept,6.001982e-09,2.160713e-07
f_t_el[T.Freq_f1_t1_el2],8.383351e-01,9.008974e-01
f_t_el[T.Freq_f1_t2_el1],7.376705e-01,8.210979e-01
f_t_el[T.Freq_f1_t2_el2],1.658357e-04,9.950144e-04
f_t_el[T.Freq_f1_t3_el1],7.684485e-06,5.029845e-05
...,...,...
V:f_t_el[T.Freq_f3_t1_el3]:congruency[T.incongr],3.109873e-01,4.975796e-01
V:f_t_el[T.Freq_f3_t2_el2]:congruency[T.incongr],4.775023e-01,6.109832e-01
V:f_t_el[T.Freq_f3_t2_el3]:congruency[T.incongr],3.286247e-01,5.034251e-01
V:f_t_el[T.Freq_f3_t3_el2]:congruency[T.incongr],2.289247e-01,4.162279e-01
